In [1]:
# Allows imports from other packages in the project
import sys
import os
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)

In [2]:
import pandas as pd

In [3]:
stocks_by_cluster_df = pd.read_csv('../data/stocks_by_cluster.csv')
stocks_by_cluster_df

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8
0,AMAT,DVN,MSFT,JPM,CVX,GOOG,PFE,TGT
1,MU,MRO,ABT,BAC,SLB,GOOGL,ABBV,TJX
2,NVDA,HAL,MGM,MS,COP,AMZN,JNJ,WBA
3,AAPL,XOM,ORCL,C,KMI,BSX,GILD,WMT
4,NFLX,MPC,EBAY,RF,F,MDLZ,BMY,CVS


## Get the data and the indicators

We collect the stock data from yfinance for between dates '2016-01-02'-'2019-01-01' (3 years) and calculate certain indicators helpful for analysis, which are:

Relative Strength Index (RSI)
Stochastic Oscillator
Williams %R
Moving Average Convergence Divergnece (MACD)
Price Rate Of Change
On Balance Volume
After calculating these indicators for each day, we add them to the dataframe of the stock as new columns. We select the columns ['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume'] at the end to obtain a new dataframe representing the stock.

We do this for each of the stocks we have selected and concatenate the individual dataframes to obtain a a resulting dataframe of shape (T, NxM), where T is the number of time steps, N the number of assets, M the number of indicators for each asset.

We also standardize the names of individual assets (e.g. asset_1) for tensorization.

Also Note that the labels are set according to the price change
 days-out (set to be 9 below).

In [4]:
from grtel.indicators import get_rsi, get_stochastic_oscillator, get_williams, get_macd, get_obv, get_indicators

In [5]:
samples = []
labels = []
for cluster in stocks_by_cluster_df.columns:
    for i, ticker in enumerate(stocks_by_cluster_df[cluster]):
        asset_name = 'asset_' + str(i+1)

        stock_data_and_indicators = get_indicators(ticker)

        X_i = stock_data_and_indicators[['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume']].copy()
        num_indicators = len(X_i.columns)
        X_i.columns = [ [cluster] * num_indicators, [asset_name] * num_indicators, X_i.columns]

        y_i = stock_data_and_indicators['Prediction'].copy()
        y_i.name = ticker

        samples.append(X_i)
        labels.append(y_i)

samples_df = pd.concat(samples, axis=1)
samples_df.columns.names = ['Cluster', 'Asset', 'Metrics']

labels_df = pd.concat(labels, axis=1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
samples_df

Cluster     cluster_1                                                       \
Asset         asset_1                                                        
Metrics         Close        RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                                         
2016-01-22  17.049999  51.119297  43.165445 -56.834555 -0.097708 -0.124555   
2016-01-25  16.910000  47.801520  38.129494 -61.870506 -0.078060 -0.114917   
2016-01-26  17.350000  57.746426  63.025232 -36.974768 -0.035484 -0.098570   
2016-01-27  17.090000  51.107526  70.454523 -29.545477 -0.018663 -0.082221   
2016-01-28  16.680000  42.266121  47.976887 -52.023113 -0.031403 -0.071871   
...               ...        ...        ...        ...       ...       ...   
2018-12-11  34.020000  39.676060  16.000019 -83.999981 -0.062314  0.159786   
2018-12-12  34.200001  41.599638  14.450871 -85.549129 -0.131203  0.101588   
2018-12-13  33.709999  37.812369   5.009603 -94.990397 -0.222768  0.036717   
2018-12-14  32.650002  30.810807   0.332786 -99.667214 -0.376527 -0.045932   
2018-12-17  32.669998  31.088556   2.450943 -97.549057 -0.491108 -0.134967   

Cluster                                                                  ...  \
Asset                                                asset_2             ...   
Metrics    Price Rate Of Change On Balance Volume      Close        RSI  ...   
Date                                                                     ...   
2016-01-22             0.015485         -82509700  11.070000  32.955810  ...   
2016-01-25             0.013789         -91721800  10.480000  27.248351  ...   
2016-01-26             0.035821         -75935100  10.550000  28.933255  ...   
2016-01-27             0.054938         -88525300  10.390000  27.267746  ...   
2016-01-28            -0.007733        -106022400   9.890000  22.580823  ...   
...                         ...               ...        ...        ...  ...   
2018-12-11            -0.055000         597071100  35.209999  37.702548  ...   
2018-12-12            -0.078416         608966600  36.029999  44.116271  ...   
2018-12-13            -0.077449         599290800  35.020000  38.485228  ...   
2018-12-14            -0.124195         588467900  34.200001  34.374934  ...   
2018-12-17            -0.147887         600730000  33.880001  32.797666  ...   

Cluster               cluster_8                                          \
Asset                   asset_4                      asset_5              
Metrics    Price Rate Of Change On Balance Volume      Close        RSI   
Date                                                                      
2016-01-22            -0.013377         120042900  95.639999  54.832811   
2016-01-25            -0.011990         158513100  94.070000  43.171986   
2016-01-26             0.005973         186836700  94.040001  42.970516   
2016-01-27             0.032784         156193800  94.339996  45.884517   
2016-01-28             0.018395         190028700  93.489998  39.316729   
...                         ...               ...        ...        ...   
2018-12-11            -0.012521         739386900  73.410004  25.598058   
2018-12-12            -0.044634         710469300  74.500000  36.029502   
2018-12-13            -0.044506         686342400  73.370003  30.854822   
2018-12-14            -0.059396         651861600  71.879997  25.321635   
2018-12-17            -0.080810         624856200  70.690002  21.730533   

Cluster                                                                    \
Asset                                                                       
Metrics     k_percent  r_percent      MACD MACD_EMA9 Price Rate Of Change   
Date                                                                        
2016-01-22  68.541642 -31.458358 -0.050732 -0.090357             0.022232   
2016-01-25  35.833336 -64.166664 -0.091800 -0.090656            -0.008955   
2016-01-26  39.486050 -60.513950 -0.1236

In [7]:
labels_df

,AMAT,MU,NVDA,AAPL,NFLX,DVN,MRO,HAL,XOM,MPC,...,PFE,ABBV,JNJ,GILD,BMY,TGT,TJX,WBA,WMT,CVS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2016-01-25,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2016-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-27,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2016-01-28,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
samples_df.to_csv('../data/samples_cluster.csv')
labels_df.to_csv('../data/labels_cluster.csv')